In [ ]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime,timezone

from configupdater import ConfigUpdater
# pip install ConfigUpdater

from dotenv import dotenv_values

from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
from google.oauth2 import service_account

In [ ]:
#dt_imported=datetime.now(timezone.utc) # utc
dt_imported=datetime.now()
dt_imported=datetime.strptime(dt_imported.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S")
print(dt_imported)

In [ ]:
view_name="pmr_project"
listIDs=[25]
key_id="project_id"


In [ ]:


projectId='kku-intern-dataai'  # smart-data-ml
dataset_id='Demo_PMReport'
table_name=view_name.replace("pmr_","")
table_id = f"{projectId}.{dataset_id}.{table_name}"
print(table_id)
to_bq_mode="WRITE_APPEND"

# Read Configuration File and Initialize BQ Object

In [ ]:
updater = ConfigUpdater()
updater.read("view_updater.cfg")

env_path='.env'
config = dotenv_values(dotenv_path=env_path)

In [ ]:
credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\kku-intern-dataai-a5449aee8483.json')
client = bigquery.Client(credentials= credentials,project=projectId)
print(client)

In [ ]:
last_imported=datetime.strptime(updater["metadata"][view_name].value,"%Y-%m-%d %H:%M:%S")
print(last_imported)

# Postgres &BigQuery

In [ ]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [ ]:
def get_bq_table():
 try:
    table=client.get_table(table_id)  # Make an API request.
    print("Table {} already exists.".format(table_id))
    print(table.schema)
    return True
 except NotFound:
    raise Exception("Table {} is not found.".format(table_id))
    
def insertDataFrameToBQ(df_trasns):
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition=to_bq_mode,
        )

        job = client.load_table_from_dataframe(
            df_trasns, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df_trasns), f"Imported data to {table_id} on bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        print(e) 



# Load view and transform

In [ ]:
if len(listIDs)>1:
 sql_view=f"select *  from {view_name}  where {key_id} in {tuple(listIDs)}"
else:
 sql_view=f"select *  from {view_name}  where {key_id} ={listIDs[0]}"
print(sql_view)
df=list_data(sql_view,None,get_postgres_conn())
df['imported_at']=dt_imported
df

# Ingest into BQ

In [ ]:
if get_bq_table():
    try:
        insertDataFrameToBQ(df)
    except Exception as ex:
        raise ex

In [ ]:
# Update Last Update

In [ ]:
updater["metadata"][view_name].value=dt_imported.strftime("%Y-%m-%d %H:%M:%S")
updater.update_file() 